In [1]:
import qutip
import numpy as np
import itertools

import sqlite3
import pickle
import time
import pandas as pd

from numpy import pi, sqrt

In [2]:
db_filename = 'SimulationRecords-ShortZ.db'

# Read simulation
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()
  
# Check if a row exists.
cursor.execute(
    '''
    SELECT * FROM Simulations
    WHERE nSteps > ?
    ''', (0,))

rows_returned = cursor.fetchall()

# Read simulation
conn = sqlite3.connect(db_filename)
sqlSelectHamiltonians = '''
    SELECT * FROM UniformHamiltonianModels
    WHERE xMagneticField < 80.0
'''
    
sqlSelectDecoherence = '''
    SELECT * FROM UniformDecoherenceModels
'''

dfHamiltonian = pd.read_sql(sqlSelectHamiltonians, con=conn)
dfDecoherence = pd.read_sql(sqlSelectDecoherence, con=conn)

In [3]:
dfHamiltonian

,uuidHamiltonian,geometry,xxInteractions,yyInteractions,zzInteractions,xMagneticField,yMagneticField,zMagneticField
0,69952101c4934670be0949d5f6301928,1d-nn,0.0,0.0,1.0,0.010000,0.0,0.0
1,ceb61c6f784d4a9fb6a01376ec6c7b9b,1d-nn,0.0,0.0,1.0,0.012409,0.0,0.0
2,7ac3ab88ab7c4aafa2796616e72f868c,1d-nn,0.0,0.0,1.0,0.015399,0.0,0.0
3,ec090c438ab3491392eae2f2da86603d,1d-nn,0.0,0.0,1.0,0.019110,0.0,0.0
4,8fb5f0e31bb64204b66ad96ff77ff66d,1d-nn,0.0,0.0,1.0,0.023714,0.0,0.0
5,aa82f80959cc476981dbf8e45d2c302b,1d-nn,0.0,0.0,1.0,0.029427,0.0,0.0
6,61d9236dfc4f417fabb9c0d355686eed,1d-nn,0.0,0.0,1.0,0.036517,0.0,0.0
7,a319601f010840efba6ab14d30b0a0fc,1d-nn,0.0,0.0,1.0,0.045316,0.0,0.0
8,0a095970598d41bca642b692030c803b,1d-nn,0.0,0.0,1.0,0.056234,0.0,0.0
9,fb84f67531614d589b79188dcdf8f73d,1d-nn,0.0,0.0,1.0,0.069783,0.0,0.0


In [4]:
dfDecoherence

,uuidDecoherence,typeDecoherence,GammaMinus,GammaPlus,GammaZ
0,8cb7e83fbd664304ab40f815144ee45a,local,0.0,0.0,0.00000
1,0e573da59297414e9a979ac95a7f6e08,local,0.0,0.0,0.00001
2,26acff8f1b644a4b97074cd71ad5cbe8,local,0.0,0.0,0.00010
3,f694b16b24c14fc4b515faac692c6aad,local,0.0,0.0,0.00100
4,af6a44c617174d14a6e8150ff57df771,local,0.0,0.0,0.01000
5,aa66d9b5b41c4634aca58d4ffec79c6d,local,0.0,0.0,0.10000


In [5]:
sqlSelectSimulations = '''
    SELECT uuidSimulations, uuidHamiltonian, uuidDecoherence, zzInteractions, xMagneticField, GammaPlus, GammaMinus
    FROM Simulations NATURAL JOIN UniformHamiltonianModels NATURAL JOIN UniformDecoherenceModels
'''

dfSimulations = pd.read_sql(sqlSelectSimulations, con=conn)

dfSimulations

,uuidSimulations,uuidHamiltonian,uuidDecoherence,zzInteractions,xMagneticField,GammaPlus,GammaMinus
0,6341da1a8fdd4648bb8e1ec23f9acbe8,69952101c4934670be0949d5f6301928,8cb7e83fbd664304ab40f815144ee45a,1.0,0.01,0.0,0.0
1,69c23783d0d84432bb5739bf9890b9d1,69952101c4934670be0949d5f6301928,0e573da59297414e9a979ac95a7f6e08,1.0,0.01,0.0,0.0
2,da4999c688894d84ae88afa8e1647ead,69952101c4934670be0949d5f6301928,26acff8f1b644a4b97074cd71ad5cbe8,1.0,0.01,0.0,0.0
3,d2c4d3520e004e60b6c6628bda5afaba,69952101c4934670be0949d5f6301928,f694b16b24c14fc4b515faac692c6aad,1.0,0.01,0.0,0.0
4,9289c1541eea48c8b505bd90360ab227,69952101c4934670be0949d5f6301928,af6a44c617174d14a6e8150ff57df771,1.0,0.01,0.0,0.0
...,...,...,...,...,...,...,...
193,4e59b4f34016403e8f75b6d75fde44dd,7d72aa677ab742d9b465e973941d4121,0e573da59297414e9a979ac95a7f6e08,1.0,10.00,0.0,0.0
194,b8700a0407644198a3bbf224c90ebb88,7d72aa677ab742d9b465e973941d4121,26acff8f1b644a4b97074cd71ad5cbe8,1.0,10.00,0.0,0.0
195,2cd2a38113ef419fa06488bf0f61a74a,7d72aa677ab742d9b465e973941d4121,f694b16b24c14fc4b515faac692c6aad,1.0,10.00,0.0,0.0
196,f9b53d53cc6b4d74b85a2a5639a422df,7d72aa677ab742d9b465e973941d4121,af6a44c617174d14a6e8150ff57df771,1.0,10.00,0.0,0.0


In [6]:
import spinsystems
def process_row (row):
    
    # Parse the data from the row
    (simulation_uuid, \
     hamiltonianmodel_uuid, \
     decoherencemodel_uuid, \
     n_spins, \
     t_initial, \
     t_final, \
     n_steps) = row
    
    print('nSpins = %d, tInitial=%g, tFinal=%g nSteps=%d' % (n_spins, t_initial, t_final, n_steps))
    # Check if the simulation has been performed
    # by querying the table `SimulationResults`
    
    cursor.execute(
        '''SELECT * FROM SimulationResults
           WHERE uuidSimulation = ?
        ''', (simulation_uuid, )
    )
    
    simulationresult_row = cursor.fetchall()
    
    if len(simulationresult_row) > 0:
        print('Simulation has already been done. Not doing again')
    else:
        print('Simulation has not been done. Running simulation.')
        # Read the model parameters
        cursor.execute(
            '''SELECT * FROM UniformHamiltonianModels
                WHERE uuidHamiltonian = ?
            ''', (hamiltonianmodel_uuid, )
        )
        hamiltonianmodel_row = cursor.fetchall()[0]

        (\
         uuidHamiltonian, \
         geometry, \
         xx_interaction, \
         yy_interaction, \
         zz_interaction, \
         x_magneticfield, \
         y_magneticfield, \
         z_magneticfield, \
        ) = hamiltonianmodel_row

        cursor.execute(
            '''SELECT * FROM UniformDecoherenceModels
                WHERE uuidDecoherence = ?
            ''', (decoherencemodel_uuid, )
        )
        decoherencemodel_row = cursor.fetchall()[0]

        (\
         uuidDecoherence, 
         decoherence_type,
         gamma_minus,
         gamma_plus,
         gamma_z
        ) = decoherencemodel_row

        # Setup model
        # :HACK: I am hard coding `theta_initial`, `phi_initial`
        theta_initial = 0
        phi_initial = 0

        n_steps = int(n_steps)

        hamiltonianmodel_object = spinsystems.TransverseFieldIsingModelNearestNeighbor1D(zz_interaction, x_magneticfield)
        decoherencemodel_object = spinsystems.SpinModelDecoherence(gamma_plus, gamma_minus, gamma_z)
        sim = spinsystems.SpinDynamicsSimulation(t_initial, t_final, n_spins, theta_initial, phi_initial, n_steps, \
                                                 hamiltonianmodel_object, decoherencemodel_object)

        # Execute model
        # :HACK: I am hard coding `n_trajectories`
        # :HACK: I am also hard coding the use of `qutip.mcsolve`
        n_trajectories=128

        timestamp_start = time.time_ns()
        sim.solve_montecarlo(n_trajectories=n_trajectories)
        timestamp_end = time.time_ns()

        # Save results to the database
        resultmc_bytes = pickle.dumps(sim.resultmc)

        cursor.execute(
            '''INSERT INTO SimulationResults
            (uuidSimulation, timeStampStart, timeStampEnd, blobResults)
            VALUES (?, ?, ?, ?)
            ''', (simulation_uuid, timestamp_start, timestamp_end, resultmc_bytes)
        )

        # Calculate

        conn.commit()

In [7]:
%%time
start_count = 0
for count, row in enumerate(rows_returned[start_count:]):
    print(time.asctime(), 'Starting row %d' %(count,))
    print(row)
    print('Processing row %d' % (start_count+count,))
    process_row(row)
    

Tue Jun  2 13:37:41 2020 Starting row 0
('6341da1a8fdd4648bb8e1ec23f9acbe8', '69952101c4934670be0949d5f6301928', '8cb7e83fbd664304ab40f815144ee45a', 8, 0, 50.26548245743669, 256)
Processing row 0
nSpins = 8, tInitial=0, tFinal=50.2655 nSteps=256
Simulation has not been done. Running simulation.
Tue Jun  2 13:37:41 2020 Starting row 1
('69c23783d0d84432bb5739bf9890b9d1', '69952101c4934670be0949d5f6301928', '0e573da59297414e9a979ac95a7f6e08', 8, 0, 50.26548245743669, 256)
Processing row 1
nSpins = 8, tInitial=0, tFinal=50.2655 nSteps=256
Simulation has not been done. Running simulation.
10.2%. Run time:   0.78s. Est. time left: 00:00:00:06
20.3%. Run time:   1.36s. Est. time left: 00:00:00:05
30.5%. Run time:   1.93s. Est. time left: 00:00:00:04
40.6%. Run time:   2.53s. Est. time left: 00:00:00:03
50.0%. Run time:   3.08s. Est. time left: 00:00:00:03
60.2%. Run time:   3.79s. Est. time left: 00:00:00:02
70.3%. Run time:   4.42s. Est. time left: 00:00:00:01
80.5%. Run time:   5.01s. Est.

# Estimating time taken to run simulations

For 961 simulations with `n_spins = 10`, the time taken
```
CPU times: user 14min 46s, sys: 8min 47s, total: 23min 34s
Wall time: 5h 29min 13s
```
It seems the time is dominated by database read/write operations.

It may be more useful to save the data to `pandas.DataFrame` and then save the `pandas.DataFrame` to an SQL table.

In [8]:
conn.close()